In [1]:
import yfinance as yf
import pandas as pd
import xlrd
import cvxpy as cp
import numpy as np
import tqdm
import logging
import warnings

# Data Procurement

## Security Procurement and Filtering

In [3]:
nasdaqlisted_df = pd.read_csv("ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt", sep="|")
nasdaqlisted_df = nasdaqlisted_df[nasdaqlisted_df["Symbol"] != "File Creation Time: 1016202521:31"]

otherlisted_df = pd.read_csv("ftp://ftp.nasdaqtrader.com/SymbolDirectory/otherlisted.txt", sep="|")
otherlisted_df = otherlisted_df[otherlisted_df["ACT Symbol"] != "File Creation Time: 1016202521:31"]
otherlisted_df["Symbol"] = otherlisted_df["ACT Symbol"]

all_df = pd.concat([nasdaqlisted_df, otherlisted_df])
all_df["Symbol"] = all_df["Symbol"].combine_first(all_df["ACT Symbol"])
all_df["Source"] = np.where(all_df["Exchange"].isna(), 'nasdaqlisted', 'otherlisted')

### Filters based on nasdaqtrader.com Symbol Directory

In [62]:
exclusions_df = pd.DataFrame({col: pd.Series(dtype=nasdaqlisted_df[col].dtype) for col in nasdaqlisted_df.columns})
exclusions_df["Reason"] = pd.Series(dtype="object")

# Exclude securities that are Test Issues, ETFs, or don't have normal Financial Status according to nasdaqlisted.

nasdaqlisted_reasons = ["Test Issue", "ETF", "Financial Status"]

for reason in nasdaqlisted_reasons:
    temp = nasdaqlisted_df[nasdaqlisted_df[reason] != "N"].copy()
    temp.loc[:, "Reason"] = reason 
    exclusions_df = pd.concat(
        [exclusions_df, temp], 
        ignore_index = True
    )

# Exclude missing symbols in nasdaqlisted

temp = nasdaqlisted_df[nasdaqlisted_df["Symbol"].isna()].copy()
temp.loc[:, "Reason"] = "Missing Symbol"
exclusions_df = pd.concat(
    [exclusions_df, temp], 
    ignore_index = True
)

# Exclude securities that are Test Issues or ETFs according to otherlisted.

otherlisted_reasons = ["Test Issue", "ETF"]

for reason in otherlisted_reasons:
    temp = otherlisted_df[otherlisted_df[reason] != "N"].copy()
    temp.loc[:, "Reason"] = reason
    exclusions_df = pd.concat(
        [exclusions_df, temp], 
        ignore_index = True
    )

# Obtain included symbols

inclusions_df = all_df[~all_df["Symbol"].isin(exclusions_df["Symbol"])]
included_symbols = set(inclusions_df["Symbol"])

### Filters based on yfinance data

In [31]:
# Fetch market cap and quote type from yfinance

logging.getLogger("yfinance").setLevel(logging.CRITICAL)
warnings.filterwarnings("ignore")

country = {}
market_cap = {}
quote_type = {}

error_symbols = []

for symbol in tqdm.tqdm(included_symbols):
    try:
        info = yf.Ticker(symbol).info
        
        country[symbol] = info.get("country")
        market_cap[symbol] = info.get("marketCap")
        quote_type[symbol] = info.get("quoteType")

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        error_symbols.append(symbol)

 43%|████▎     | 3004/6956 [26:22<16:31:20, 15.05s/it]

Error fetching data for ALZN: Failed to perform, curl: (28) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


 82%|████████▏ | 5690/6956 [53:41<5:17:19, 15.04s/it] 

Error fetching data for GCTS.W: Failed to perform, curl: (28) Recv failure: Connection was reset. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.


100%|██████████| 6956/6956 [1:04:38<00:00,  1.79it/s]


In [32]:
# Retry for symbols that raised errors

for symbol in tqdm.tqdm(error_symbols):
    try:
        info = yf.Ticker(symbol).info
        
        country[symbol] = info.get("country")
        market_cap[symbol] = info.get("marketCap")
        quote_type[symbol] = info.get("quoteType")

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")

100%|██████████| 2/2 [00:01<00:00,  1.62it/s]


In [63]:
# Add fetched data to inclusions_df

output_df = pd.DataFrame({
    "Country": country,
    "Market Cap": market_cap,
    "Quote Type": quote_type,
})
output_df = output_df.reset_index().rename(columns = {"index" : "Symbol"})
output_df.to_parquet("data/processed/yfinance_output.parquet", engine = "fastparquet") # External saving in case of catastrophe

inclusions_df = inclusions_df.merge(output_df, on='Symbol', how='left')

In [64]:
# Exclude securities that are not listed as EQUITY on yfinance

temp = inclusions_df[inclusions_df["Quote Type"] != "EQUITY"].copy()
temp.loc[:, "Reason"] = "Not Equity"
exclusions_df = pd.concat(
    [exclusions_df, temp], 
    ignore_index = True
)

inclusions_df = inclusions_df[~inclusions_df["Symbol"].isin(exclusions_df["Symbol"])]

# Exclude securities that have no listed market cap on yfinance

temp = inclusions_df[inclusions_df["Market Cap"].isna()].copy()
temp.loc[:, "Reason"] = "No Market Cap"
exclusions_df = pd.concat(
    [exclusions_df, temp], 
    ignore_index = True
)

inclusions_df = inclusions_df[~inclusions_df["Symbol"].isin(exclusions_df["Symbol"])]

# Exclude securities not in the US (for US index replication)

temp = inclusions_df[inclusions_df["Country"] != "United States"].copy()
temp.loc[:, "Reason"] = "Not United States"
exclusions_df = pd.concat(
    [exclusions_df, temp], 
    ignore_index = True
)

inclusions_df = inclusions_df[~inclusions_df["Symbol"].isin(exclusions_df["Symbol"])]

### Filters based on name filtering

Nasdaqlisted securities follow the convention Name - Description. Regex matching is used on the description to identify and filter out keywords (units, fund, depositary).

In [65]:
# Exclude units, funds and depositary shares

keywords = ["Unit", "Units", "Fund", "Depositary", "Warrant"]

for keyword in keywords:
    temp = inclusions_df[inclusions_df["Security Name"].str.split(" - ", n=1, expand=True)[1].fillna("").str.contains(rf"\b({keyword})\b", case=False, regex=True)]
    temp.loc[:, "Reason"] = f"Keyword found: {keyword}"
    exclusions_df = pd.concat(
        [exclusions_df, temp], 
        ignore_index = True
    )
    inclusions_df = inclusions_df[~inclusions_df["Symbol"].isin(exclusions_df["Symbol"])]

In [66]:
# Final filtering results

print("All securities:", all_df.shape[0])
print("Excluded securities:", exclusions_df.shape[0])
print("Included securities:", inclusions_df.shape[0])
print("\nExclusion reasons:\n", exclusions_df["Reason"].value_counts())

All securities: 11882
Excluded securities: 7600
Included securities: 4294

Exclusion reasons:
 Reason
ETF                          4591
Not United States            1317
No Market Cap                 747
Not Equity                    522
Financial Status              310
Test Issue                     36
Keyword found: Fund            31
Keyword found: Unit            15
Keyword found: Depositary      13
Keyword found: Units           13
Keyword found: Warrant          4
Missing Symbol                  1
Name: count, dtype: int64


In [67]:
# Export filtering results for future reference

exclusions_df["Source"] = np.where(exclusions_df["Exchange"].isna(), 'nasdaqlisted', 'otherlisted') # Fill missing sources values

exclusions_df.to_parquet("data/processed/exclusions.parquet", engine = "fastparquet")
inclusions_df.to_parquet("data/processed/inclusions.parquet", engine = "fastparquet")

In [2]:
inclusions_df = pd.read_parquet("data/processed/inclusions.parquet", engine = "fastparquet")

## Daily Price Procurement

In [4]:
# Fetch daily price history from yahoo finance

included_symbols = set(inclusions_df["Symbol"])
daily_df = yf.download(
    tickers=included_symbols,
    start="2021-10-08",
    end="2025-10-08",
    interval="1d",
    group_by='ticker',
    auto_adjust=True,
    progress=True,
    threads=False
)

[*********************100%***********************]  4294 of 4294 completed

8 Failed downloads:
['MLACR', 'BKHAR', 'HVIIR', 'MAYAR', 'DMAAR']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-10-08 -> 2025-10-08)')
['ALH']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
['TLNC', 'TTRX']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-10-08 -> 2025-10-08) (Yahoo error = "Data doesn\'t exist for startDate = 1633665600, endDate = 1759896000")')


In [11]:
# Export with date as column, keeping only close for efficiency
daily_df = daily_df[[col for col in daily_df.columns if col[1] == 'Close']]
daily_df = daily_df.reset_index()
daily_df.to_parquet("data/processed/daily_data.parquet", engine="fastparquet")